In [1]:
import os
import math
import pandas as pd
import torch
import numpy as np
from torch import nn
from torchvision import models
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchsummary import summary
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
def indexToLabel(idx):
  map = {
      0 : 'AnnualCrop',
      1 : 'Forest',
      2 : 'HerbaceousVegatation',
      3 : 'Highway',
      4 : 'Industrial',
      5 : 'Pasture',
      6 : 'PermanentCrop',
      7 : 'Residential',
      8 : 'River',
      9 : 'SeaLake'
  }
  for key in map:
    if key == idx:
      return map[key]

In [3]:
class customDataset(Dataset):
    def __init__(self, dataFilePath, csvFilePath, transform = None):
        csv = pd.read_csv(csvFilePath)
        # csv = csv.iloc[:6400]
        self.imgPathList = csv['Filename'].tolist()
        self.images = [os.path.join(dataFilePath,img) for img in self.imgPathList]
        self.labels = csv['Label'].tolist()
        self.transform = transform
        # self.labels = self.labels[:6400]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self,idx):
        imgPath = self.images[idx]
        image = read_image(imgPath)
        image = image.type(torch.float32)
        label = self.labels[idx]
        if self.transform is not None:
          image = self.transform(image)
        return image, label

In [4]:
trainData = customDataset('/kaggle/input/eurosat/2750','/kaggle/input/csv-data/train.csv')
trainDataloader = DataLoader(trainData,batch_size = 64)
valData = customDataset('/kaggle/input/eurosat/2750','/kaggle/input/csv-data/validation.csv')
valDataloader = DataLoader(valData,batch_size = 64)

In [5]:
def init_weights(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        torch.nn.init.kaiming_uniform_(m.weight)

class idBlock(nn.Module):
  def __init__(self,inChannel,filters,kernelSize):
    super().__init__()
    self.inChannel = inChannel
    self.kernelSize = kernelSize
    self.f1, self.f2, self.f3 = filters
    self.relu = nn.ReLU()
    self.conv1 = nn.Sequential(
        nn.Conv2d(self.inChannel, self.f1, 1),
        nn.BatchNorm2d(self.f1),
        nn.ReLU())
    self.conv2 = nn.Sequential(
        nn.Conv2d(self.f1,self.f2,self.kernelSize,padding = 'same'),
        nn.BatchNorm2d(self.f2),
        nn.ReLU())
    self.conv3 = nn.Sequential(
        nn.Conv2d(self.f2,self.f3,1),
        nn.BatchNorm2d(self.f3))
  def forward(self,x):
    xShortcut = x
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = torch.add(x,xShortcut)
    x = self.relu(x)
    return x

class convBlock(nn.Module):
  def __init__(self,inChannel,filters,kernelSize,stride = 2):
    super().__init__()
    self.inChannel = inChannel
    self.kernelSize = kernelSize
    self.stride = stride
    self.relu = nn.ReLU()
    self.f1, self.f2, self.f3 = filters
    self.conv1 = nn.Sequential(
        nn.Conv2d(self.inChannel, self.f1, 1, self.stride),
        nn.BatchNorm2d(self.f1),
        nn.ReLU())
    self.conv2 = nn.Sequential(
        nn.Conv2d(self.f1,self.f2,self.kernelSize,padding = 'same'),
        nn.BatchNorm2d(self.f2),
        nn.ReLU())
    self.conv3 = nn.Sequential(
        nn.Conv2d(self.f2,self.f3,1),
        nn.BatchNorm2d(self.f3))
    self.convShort = nn.Sequential(
        nn.Conv2d(self.inChannel,self.f3,1,self.stride),
        nn.BatchNorm2d(self.f3))

  def forward(self,x):
    xShortcut = self.convShort(x)
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = torch.add(x,xShortcut)
    x = self.relu(x)
    return x

class resNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.ZeroPad2d(3),
            nn.Conv2d(3,64,7,2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3,2)
        )
        self.network.apply(init_weights)
        self.layer1 = nn.Sequential(
            convBlock(64,[64, 64, 256],3, stride=1),
            idBlock(256, [64, 64, 256],3),
            idBlock(256, [64, 64, 256],3)
        )
        self.layer1.apply(init_weights)
        self.layer2 = nn.Sequential(
            convBlock(256,[128,128,512],3),
            idBlock(512, [128, 128, 512],3),
            idBlock(512, [128, 128, 512],3),
            idBlock(512, [128, 128, 512],3)
        )
        self.layer2.apply(init_weights)
        self.layer3 = nn.Sequential(
            convBlock(512,[256,256,1024],3),
            idBlock(1024, [256,256,1024],3),
            idBlock(1024, [256,256,1024],3),
            idBlock(1024, [256,256,1024],3),
            idBlock(1024, [256,256,1024],3),
            idBlock(1024, [256,256,1024],3)
        )
        self.layer3.apply(init_weights)
        self.layer4 = nn.Sequential(
            convBlock(1024,[512, 512, 2048],3),
            idBlock(2048, [512, 512, 2048],3),
            idBlock(2048, [512, 512, 2048],3)
        )
        self.layer4.apply(init_weights)
        self.avgPool = nn.AvgPool2d(2)
        self.flatten = nn.Flatten()
        self.flatten.apply(init_weights)
        self.linear = nn.Linear(2048,10)
        self.linear.apply(init_weights)

    def forward(self,x):
      x = self.network(x)
      x = self.layer1(x)
      x = self.layer2(x)
      x = self.layer3(x)
      x = self.layer4(x)
      x = self.avgPool(x)
      x = self.flatten(x)
      x = self.linear(x)
      return x

In [7]:
def train(dataloader, model, lossFn, optimizer):
  model.train()
  size = len(dataloader.dataset)
  batchSize = len(dataloader)
  numOfBatch = 0
  divider = math.ceil(batchSize/10)
  for batch, (X,y) in enumerate(dataloader):
    X = X.to(device)
    y = y.to(device)
    pred = model(X)
    loss = lossFn(pred,y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    numOfBatch += 1

    if numOfBatch % divider == 0:
        loss = loss.item()
        print(f"loss: {loss}  [{numOfBatch}/{batchSize}]")

In [8]:
def test(dataloader, model, lossFn):
  model.eval()
  size = len(dataloader.dataset)
  loss, correct = 0, 0
  with torch.no_grad():
    for batch, (X,y) in enumerate(dataloader):
      X = X.to(device)
      y = y.to(device)
      pred = model(X)
      loss = lossFn(pred,y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    loss /= 64
    correct /= size
    print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")

In [16]:
model = resNet()
model = model.to(device)
lossFn = nn.CrossEntropyLoss()

In [22]:
optimizer = torch.optim.Adam(model.parameters(),0.0001)

In [23]:
epochs = 10
for i in range(epochs):
  print(f"\nEpoch {i+1}\n-------------------------------")
  train(trainDataloader, model, lossFn, optimizer)
  print('\nValidation Accuracy: ')
  test(valDataloader, model, lossFn)


Epoch 1
-------------------------------
loss: 0.000713702233042568  [30/296]
loss: 0.007954570464789867  [60/296]
loss: 0.003219564212486148  [90/296]
loss: 0.0020873877219855785  [120/296]
loss: 0.004033444449305534  [150/296]
loss: 0.0015910614747554064  [180/296]
loss: 0.0023551557678729296  [210/296]
loss: 0.005160483531653881  [240/296]
loss: 0.001904813339933753  [270/296]

Validation Accuracy: 
Accuracy: 90.4%, Avg loss: 0.002014 


Epoch 2
-------------------------------
loss: 0.0006964280037209392  [30/296]
loss: 0.003123322268947959  [60/296]
loss: 0.001574733410961926  [90/296]
loss: 0.0008187611820176244  [120/296]
loss: 0.0017325326334685087  [150/296]
loss: 0.0007670306949876249  [180/296]
loss: 0.0009521341999061406  [210/296]
loss: 0.0010233237408101559  [240/296]
loss: 0.0014452059986069798  [270/296]

Validation Accuracy: 
Accuracy: 90.4%, Avg loss: 0.001748 


Epoch 3
-------------------------------
loss: 0.0007216868689283729  [30/296]
loss: 0.001104450086131692  [

In [18]:
testData = customDataset('/kaggle/input/eurosat/2750','/kaggle/input/csv-data/test.csv')
testDataloader = DataLoader(testData,batch_size = 64)

In [24]:
  test(testDataloader, model, lossFn)

Accuracy: 91.2%, Avg loss: 0.019985 

